In [16]:
# Initializing stuff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
file_name = "ignore_dir/whatsapp_stats.json"
all_data_df = pd.read_json(file_name)
all_data_df.sample(10)

,author,date,dialog,dialog_type
6169,Aaditya,2016-03-07 15:04:00,Nikalra,default
1637,Aakash,2016-01-25 21:01:00,Ha phir,default
20,Abhishek,2015-01-11 12:10:00,Tower ?,default
6069,Aaditya,2016-03-07 13:40:00,Ravi teja? Takla Anil?,default
1484,Aakash,2016-04-01 21:42:00,Monday tum log wapis,default
2200,Aakash,2016-02-14 11:47:00,Woh kya mila be,default
1047,Abhishek,2015-11-21 11:19:00,Uncle's Chinese final hai.,default
4866,Ravi,2016-11-06 10:48:00,aadi bhau....,default
2641,Aakash,2016-06-03 20:01:00,😘😘,default
6118,Aaditya,2016-03-07 13:56:00,Woh hai na bhau 😒,default


In [34]:
# Getting only the chat data, not the notif/message etc.
chat_df = all_data_df[all_data_df['dialog_type'] == 'default']
del chat_df['dialog_type']
chat_df.head(10)

,author,date,dialog
2,Aakash,2015-09-10 21:13:00,<Media omitted>
3,Abhishek,2015-09-10 23:49:00,Kal ka plan karo
4,Bhargav,2015-10-13 21:00:00,http://m.shopclues.com/cow-dung-cakes-gobar-up...
5,Bhargav,2015-10-13 21:04:00,Sab kuch online milta hai bc
6,Abhishek,2015-10-18 01:32:00,<Media omitted>
7,Aakash,2015-10-24 13:17:00,<Media omitted>
8,Bhargav,2015-10-24 13:45:00,Bhai aadi ko maro sabse phele bc phir jaha jan...
9,Bhargav,2015-10-24 13:46:00,Ab toh woh chala jayega phir
10,Bhargav,2015-10-24 13:46:00,Agle 5 se 6 years ke bday bumps bhi sath mein
11,Bhargav,2015-10-24 13:47:00,Ha wahi bc gmat dene ki jarurat nahi hai
